## Calculate the nearest subway station

In [6]:
import numpy as np
import pandas as pd
import geopandas as gpd

from tqdm import tqdm
tqdm.pandas(desc="Applying function")

from shapely.geometry import Point

In [7]:
# currently we sampled entire dataset due to submit
gdf_building = gpd.read_file('../../data/processed/building/building_sample.geojson')

In [8]:
gdf_building

,heightroof,mpluto_bbl,cnstrct_yr,globalid,bin,shape_len,shape_area,OfficeArea,RetailArea,ResArea,OBJECTID,StreetWidth_Min,StreetWidth_Max,POSTED_SPEED,betweeness,geometry
0,55.89,1006100054,1890,{3AEADBE0-CC54-4D24-B238-035EACC2FCA7},1010689,0.0,636520502.758,0,1050,8294,104502,24.0,25.0,25,0.033791,"MULTIPOLYGON (((984248.206 206864.602, 984228...."
1,43.48,1014390003,1920,{16E3A900-8D55-4377-ABC9-3CFEAD5AABEA},1044690,None,636520502.758,0,2175,6525,99738,60.0,60.0,25,0.033479,POINT (994870.595 217574.392)
2,164.86,1011480001,1925,{A569DF64-C060-4E22-A45B-C3AAF3994CF0},1030169,0.0,636520502.758,0,5000,118840,94940,40.0,42.0,25,0.005477,"MULTIPOLYGON (((989996.502 223961.988, 989991...."
3,57.41,1019660033,1901,{FF1D608A-A8F9-4BED-8330-01D6E785BBDE},1084099,0.0,636520502.758,0,3500,15130,97885,30.0,35.0,25,0.020543,"MULTIPOLYGON (((996485.007 235709.202, 996487...."
4,80.0,1019880018,2017,{F3EEBE92-794C-4F0B-914A-86A784D1532B},1089415,0.0,636520502.758,2962,0,16905,107246,50.0,52.0,25,0.066931,"MULTIPOLYGON (((996978.357 237617.591, 997012...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5728,66.64,1022380001,1952,{229CCEE9-7A0D-4559-B302-429BC53E7ECE},1064950,0.0,636520502.758,1000,0,61964,103391,30.0,30.0,20,0.003432,"MULTIPOLYGON (((1004982.588 255225.951, 100499..."
5729,77.46,1002030010,1890,{8CEF68E8-47D9-4D24-817F-E25BDE985A47},1077584,0.0,636520502.758,0,590,22664,92269,70.0,72.0,25,0.099612,"MULTIPOLYGON (((985266.290 200606.727, 985267...."
5730,47.07,1015300028,1920,{FA55A8C7-2668-42F4-A6CE-AA4142E24646},1048743,0.0,636520502.758,0,2282,6482,118672,70.0,70.0,25,0.029023,"MULTIPOLYGON (((997407.552 222407.276, 997364...."
5731,76.1,1004820002,1888,{EDB5257F-422A-46F8-AD16-1650AB007E29},1007207,0.0,636520502.758,0,2075,9445,95670,46.0,46.0,25,0.047275,"MULTIPOLYGON (((984709.679 202192.621, 984696...."


In [9]:
gdf_subway_ridership_2023 = gpd.read_file('../../data/processed/traffic/2023_ridership.geojson').to_crs(2263)
gdf_subway_ridership_2022 = gpd.read_file('../../data/processed/traffic/2022_ridership.geojson').to_crs(2263)

In [10]:
# spatial join between building and station
gdf_building_ridership_2023 = gdf_building.sjoin_nearest(gdf_subway_ridership_2023, distance_col='distance_from_station(ft)').drop('index_right', axis=1)

# because duplicated rows created during the spatial join process, I dropped
gdf_building_ridership_2023 = gdf_building_ridership_2023.drop_duplicates(subset=['bin','mpluto_bbl','globalid'])

In [11]:
# spatial join between building and station
gdf_building_ridership_2022 = gdf_building.sjoin_nearest(gdf_subway_ridership_2022, distance_col='distance_from_station(ft)').drop('index_right', axis=1)

# because duplicated rows created during the spatial join process, I dropped
gdf_building_ridership_2022 = gdf_building_ridership_2022.drop_duplicates(subset=['bin','mpluto_bbl','globalid'])

gdf_building = gdf_building_ridership_2023.copy()

In [12]:
# export dataset for the prediction

gdf_building_ridership_2023 = gdf_building_ridership_2023.loc[:,['bin','ridership_evening','ridership_late_night','ridership_midday','ridership_morning','ridership_night','distance_from_station(ft)']]
gdf_building_ridership_2022 = gdf_building_ridership_2022.loc[:,['bin','ridership_evening','ridership_late_night','ridership_midday','ridership_morning','ridership_night','distance_from_station(ft)']]

gdf_building_ridership_2023.loc[:,'year'] = 2023
gdf_building_ridership_2022.loc[:,'year'] = 2022

gdf_building_ridership = pd.concat([gdf_building_ridership_2023, gdf_building_ridership_2022], ignore_index=True)

# gdf_building_ridership.to_csv('../../data/processed/traffic/ridership_by_bin.csv', index=False)

## Mapping AADT 2019 ~ 2021

Estimated AADT values for each building were calculated as the inverse distance weighted average 

In [13]:
gdf_aadt_2019 = gpd.read_file('../../data/raw/aadt/aadt_2019.geojson').to_crs(2263)
gdf_aadt_2020 = gpd.read_file('../../data/raw/aadt/aadt_2020.geojson').to_crs(2263)
gdf_aadt_2021 = gpd.read_file('../../data/raw/aadt/aadt_2021.geojson').to_crs(2263)


gdf_aadt_2019 =  gdf_aadt_2019.loc[np.logical_not(gdf_aadt_2019.loc[:,'AADT'].isnull())]
gdf_aadt_2020 =  gdf_aadt_2020.loc[np.logical_not(gdf_aadt_2020.loc[:,'MAT_ALH_2020_csv_AADT'].isnull())].rename(columns={'MAT_ALH_2020_csv_AADT':'AADT'})
gdf_aadt_2021 =  gdf_aadt_2021.loc[np.logical_not(gdf_aadt_2021.loc[:,'MAT_ALH_2021_csv_AADT'].isnull())].rename(columns={'MAT_ALH_2021_csv_AADT':'AADT'})

In [14]:
def calculate_idw_aadt_average(building, gdf_aadt):
    
    distances = gdf_aadt.distance(building)
    inverse_distance_weighted_traffic = np.average(gdf_aadt['AADT'], weights = 1/(distances + 1))

    return inverse_distance_weighted_traffic

In [15]:
gdf_building.loc[:,'idw_aadt_2019'] = gdf_building.loc[:,'geometry'].progress_apply(lambda x: calculate_idw_aadt_average(x, gdf_aadt_2019))

Applying function: 100%|██████████| 5733/5733 [03:05<00:00, 30.89it/s]


In [16]:
gdf_building.loc[:,'idw_aadt_2020'] = gdf_building.loc[:,'geometry'].progress_apply(lambda x: calculate_idw_aadt_average(x, gdf_aadt_2020))

Applying function: 100%|██████████| 5733/5733 [01:06<00:00, 85.76it/s] 


In [17]:
gdf_building.loc[:,'idw_aadt_2021'] = gdf_building.loc[:,'geometry'].progress_apply(lambda x: calculate_idw_aadt_average(x, gdf_aadt_2021))

Applying function: 100%|██████████| 5733/5733 [00:58<00:00, 97.69it/s] 


In [18]:
gdf_building.head()

,heightroof,mpluto_bbl,cnstrct_yr,globalid,bin,shape_len,shape_area,OfficeArea,RetailArea,ResArea,...,station_complex_id,ridership_evening,ridership_late_night,ridership_midday,ridership_morning,ridership_night,distance_from_station(ft),idw_aadt_2019,idw_aadt_2020,idw_aadt_2021
0,55.89,1006100054,1890,{3AEADBE0-CC54-4D24-B238-035EACC2FCA7},1010689,0.0,636520502.758,0,1050,8294,...,167,1039.193346,246.533377,817.890835,318.025307,843.734048,748.256620,7193.081669,7412.369895,8350.144642
1,43.48,1014390003,1920,{16E3A900-8D55-4377-ABC9-3CFEAD5AABEA},1044690,None,636520502.758,0,2175,6525,...,223,464.188485,32.428631,391.078729,172.856077,159.304594,1264.066674,12988.853950,12048.318975,13486.718605
2,164.86,1011480001,1925,{A569DF64-C060-4E22-A45B-C3AAF3994CF0},1030169,0.0,636520502.758,0,5000,118840,...,312,374.109700,25.401808,389.797037,207.302407,141.154191,920.880326,13677.792850,13732.269067,15411.109097
3,57.41,1019660033,1901,{FF1D608A-A8F9-4BED-8330-01D6E785BBDE},1084099,0.0,636520502.758,0,3500,15130,...,306,212.837888,22.964786,228.687483,124.355940,87.609534,987.480526,11736.070949,12900.752001,14449.074278
4,80.0,1019880018,2017,{F3EEBE92-794C-4F0B-914A-86A784D1532B},1089415,0.0,636520502.758,2962,0,16905,...,305,210.033462,44.356785,343.021296,269.631282,71.344027,1085.327457,12835.701327,14143.073155,15793.074614


In [19]:
gdf_building_renamed = gdf_building.rename(columns={'heightroof':'height',
                                            'OfficeArea':'office_area',
                                            'RetailArea':'retail_area',
                                            'ResArea':'residential_area',
                                            'StreetWidth_Min':'street_width_min',
                                            'StreetWidth_Max':'street_width_max',
                                            'POSTED_SPEED':'posted_speed'})

## Mapping ATVC 2018 ~ 2020

In [20]:
# vehicle count.geojson was generated by QGIS based on data from traffic_count.csv which was created by traffic_count_data.ipynb
gdf_atvc = gpd.read_file('../../data/raw/traffic_count/vehicle count.geojson').to_crs(2263)
gdf_atvc = gdf_atvc.loc[gdf_atvc.loc[:,'Yr']<2022]

In [21]:
def calculate_idw_atvc_average(building, gdf_atvc, year):
    gdf_tmp = gdf_atvc.loc[gdf_atvc.loc[:,'Yr']==year]
    distances = gdf_tmp.distance(building)
    inverse_distance_weighted_traffic = np.average(gdf_tmp['median'], weights = 1/(distances + 1))

    return inverse_distance_weighted_traffic

In [22]:
gdf_building_renamed.loc[:,'idw_atvc_2018'] = gdf_building_renamed.loc[:,'geometry'].progress_apply(lambda x: calculate_idw_atvc_average(x, gdf_atvc, 2018))

Applying function: 100%|██████████| 5733/5733 [00:05<00:00, 1052.30it/s]


In [23]:
gdf_building_renamed.loc[:,'idw_atvc_2019'] = gdf_building_renamed.loc[:,'geometry'].progress_apply(lambda x: calculate_idw_atvc_average(x, gdf_atvc, 2019))

Applying function: 100%|██████████| 5733/5733 [00:06<00:00, 937.51it/s]


In [24]:
gdf_building_renamed.loc[:,'idw_atvc_2020'] = gdf_building_renamed.loc[:,'geometry'].progress_apply(lambda x: calculate_idw_atvc_average(x, gdf_atvc, 2020))

Applying function: 100%|██████████| 5733/5733 [00:03<00:00, 1803.41it/s]


In [25]:
# export vehicle count data for the prediction
df_vehicle_atvc = gdf_building_renamed.loc[:,['bin','idw_atvc_2018','idw_atvc_2019','idw_atvc_2020']].set_index('bin').stack().reset_index()
df_vehicle_atvc = df_vehicle_atvc.rename(columns={'level_1':'year', 0:'idw_atvc'})
df_vehicle_atvc.loc[:,'year'] = df_vehicle_atvc.loc[:,'year'].str.replace('idw_atvc_','').astype(int)
df_vehicle_atvc.to_csv('../../data/processed/traffic/idw_atvc_by_bin.csv', index=False)

In [26]:
# export vehicle count data for the prediction
df_vehicle_aadt = gdf_building_renamed.loc[:,['bin','idw_aadt_2019','idw_aadt_2020','idw_aadt_2021']].set_index('bin').stack().reset_index()
df_vehicle_aadt = df_vehicle_aadt.rename(columns={'level_1':'year', 0:'idw_aadt'})
df_vehicle_aadt.loc[:,'year'] = df_vehicle_aadt.loc[:,'year'].str.replace('idw_aadt_','').astype(int)
df_vehicle_aadt.to_csv('../../data/processed/traffic/idw_aadt_by_bin.csv', index=False)

In [27]:
gdf_building_renamed.to_file('../../data/processed/building/building_sample.geojson', driver='GeoJSON')